# Importance-Weighted Training with Adversarial Validation

**Hypothesis**: The CV-LB gap suggests distribution shift between train and test. Importance weighting reweights training instances to match the test distribution, which could fundamentally change the CV-LB relationship.

**Key Insight from Analysis**:
- CV-LB relationship: LB = 4.22×CV + 0.0534 (R²=0.98)
- Intercept (0.0534) > Target (0.0347) - CV minimization alone CANNOT reach target
- We need an approach that CHANGES the CV-LB relationship

**Implementation**:
1. For each fold, use adversarial validation to identify distribution shift
2. Train a classifier to distinguish train vs test samples using solvent features
3. Use the classifier's probability as importance weights
4. Apply weighted loss during training

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.ensemble import RandomForestClassifier
import tqdm
import warnings
warnings.filterwarnings('ignore')

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

# Simple Featurizer (for GP) - 18 features
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')
print(f'Simple feature dimension: {SimpleFeaturizer().feats_dim}')

Full feature dimension: 145
Simple feature dimension: 18


In [5]:
# MLP Model with weighted loss
class MLPModel(nn.Module):
    def __init__(self, input_dim=145, hidden_dim=64, output_dim=3, dropout=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

print('MLP model defined')

MLP model defined


In [6]:
# Importance-Weighted Ensemble Model
class ImportanceWeightedEnsemble:
    """
    GP + MLP + LGBM ensemble with importance weighting.
    
    Uses adversarial validation to compute importance weights that
    reweight training samples to match the test distribution.
    """
    def __init__(self, data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.30):
        self.data = data
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        self.mixed = (data == 'full')
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        self.gp_models = []
        self.mlp_model = None
        self.lgbm_models = []
        
    def compute_importance_weights(self, X_train_feat, X_test_feat):
        """
        Compute importance weights using adversarial validation.
        
        Train a classifier to distinguish train vs test samples.
        The importance weight for a training sample is p(test|x) / p(train|x).
        """
        # Create labels: 0 for train, 1 for test
        n_train = len(X_train_feat)
        n_test = len(X_test_feat)
        
        X_combined = np.vstack([X_train_feat, X_test_feat])
        y_combined = np.concatenate([np.zeros(n_train), np.ones(n_test)])
        
        # Train a classifier
        clf = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
        clf.fit(X_combined, y_combined)
        
        # Get probability of being test for training samples
        p_test = clf.predict_proba(X_train_feat)[:, 1]
        
        # Importance weight: p(test|x) / p(train|x) = p_test / (1 - p_test)
        # Clip to avoid division by zero and extreme weights
        p_test = np.clip(p_test, 0.01, 0.99)
        weights = p_test / (1 - p_test)
        
        # Normalize weights to sum to n_train
        weights = weights * n_train / weights.sum()
        
        return weights
        
    def train_model(self, X, Y, X_test=None):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        
        # Compute importance weights if test data is provided
        if X_test is not None:
            X_test_full = self.full_featurizer.featurize(X_test)
            X_test_full_scaled = self.scaler_full.transform(X_test_full)
            weights = self.compute_importance_weights(X_full_scaled, X_test_full_scaled)
        else:
            weights = np.ones(len(X_full_scaled))
        
        # Train GP for each target (GP doesn't support sample weights directly)
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(
                kernel=kernel, alpha=0.01, n_restarts_optimizer=3, random_state=42
            )
            gp.fit(X_simple_scaled, Y_np[:, i])
            self.gp_models.append(gp)
        
        # Train MLP with weighted loss
        self.mlp_model = MLPModel(
            input_dim=self.full_featurizer.feats_dim,
            hidden_dim=64,
            output_dim=3,
            dropout=0.2
        ).to(device)
        
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_np, dtype=torch.double).to(device)
        weights_tensor = torch.tensor(weights, dtype=torch.double).to(device)
        
        optimizer = torch.optim.Adam(self.mlp_model.parameters(), lr=0.001, weight_decay=1e-4)
        
        self.mlp_model.train()
        for epoch in range(200):
            optimizer.zero_grad()
            pred = self.mlp_model(X_tensor)
            # Weighted MSE loss
            loss = (weights_tensor.unsqueeze(1) * (pred - Y_tensor) ** 2).mean()
            loss.backward()
            optimizer.step()
        
        # Train LGBM for each target with sample weights
        for i in range(3):
            lgbm = lgb.LGBMRegressor(
                n_estimators=100, learning_rate=0.05, max_depth=5,
                num_leaves=31, min_child_samples=5, random_state=42, verbose=-1
            )
            lgbm.fit(X_full_scaled, Y_np[:, i], sample_weight=weights)
            self.lgbm_models.append(lgbm)
        
        return self
    
    def predict(self, X):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # GP predictions
        gp_preds = np.column_stack([self.gp_models[i].predict(X_simple_scaled) for i in range(3)])
        
        # MLP predictions
        self.mlp_model.eval()
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        with torch.no_grad():
            mlp_preds = self.mlp_model(X_tensor).cpu().numpy()
        
        # LGBM predictions
        lgbm_preds = np.column_stack([self.lgbm_models[i].predict(X_full_scaled) for i in range(3)])
        
        # Weighted ensemble
        predictions = self.gp_weight * gp_preds + self.mlp_weight * mlp_preds + self.lgbm_weight * lgbm_preds
        
        # TTA for mixtures
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
            
            X_full_scaled_flip = self.scaler_full.transform(X_full_flip)
            X_simple_scaled_flip = self.scaler_simple.transform(X_simple_flip)
            
            gp_preds_flip = np.column_stack([self.gp_models[i].predict(X_simple_scaled_flip) for i in range(3)])
            X_tensor_flip = torch.tensor(X_full_scaled_flip, dtype=torch.double).to(device)
            with torch.no_grad():
                mlp_preds_flip = self.mlp_model(X_tensor_flip).cpu().numpy()
            lgbm_preds_flip = np.column_stack([self.lgbm_models[i].predict(X_full_scaled_flip) for i in range(3)])
            
            predictions_flip = self.gp_weight * gp_preds_flip + self.mlp_weight * mlp_preds_flip + self.lgbm_weight * lgbm_preds_flip
            predictions = (predictions + predictions_flip) / 2
        
        predictions = np.clip(predictions, 0, 1)
        return torch.tensor(predictions)

print('Importance-Weighted Ensemble defined')

Importance-Weighted Ensemble defined


In [7]:
# Quick test
X_single, Y_single = load_data("single_solvent")
print(f'Single solvent data: X={X_single.shape}, Y={Y_single.shape}')

# Test on a small subset
X_train = X_single.iloc[:80]
Y_train = Y_single.iloc[:80]
X_test = X_single.iloc[80:100]
Y_test = Y_single.iloc[80:100]

model = ImportanceWeightedEnsemble(data='single')
model.train_model(X_train, Y_train, X_test=X_test)
preds = model.predict(X_test)
print(f'Test predictions shape: {preds.shape}')
print(f'Test predictions range: [{preds.min():.4f}, {preds.max():.4f}]')

Single solvent data: X=(656, 3), Y=(656, 3)


Test predictions shape: torch.Size([20, 3])
Test predictions range: [0.0908, 0.7229]


In [8]:
# Run CV on single solvent data
print('\n=== Single Solvent CV (Importance-Weighted) ===')
X_single, Y_single = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = ImportanceWeightedEnsemble(data='single')
    model.train_model(train_X, train_Y, X_test=test_X)
    predictions = model.predict(test_X)
    
    all_predictions_single.append(predictions.numpy())
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_predictions_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((preds_single - actuals_single) ** 2)
print(f'Single Solvent MSE: {mse_single:.6f} (n={len(preds_single)})')


=== Single Solvent CV (Importance-Weighted) ===


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:29<11:21, 29.63s/it]

  8%|▊         | 2/24 [00:58<10:39, 29.05s/it]

 12%|█▎        | 3/24 [01:20<09:00, 25.73s/it]

...

 92%|█████████▏| 22/24 [10:21<00:56, 28.34s/it]

 96%|█████████▌| 23/24 [10:53<00:29, 29.47s/it]

100%|██████████| 24/24 [11:20<00:00, 28.81s/it]

100%|██████████| 24/24 [11:20<00:00, 28.36s/it]

Single Solvent MSE: 0.009723 (n=656)


In [9]:
# Run CV on full data
print('\n=== Full Data CV (Importance-Weighted) ===')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = ImportanceWeightedEnsemble(data='full')
    model.train_model(train_X, train_Y, X_test=test_X)
    predictions = model.predict(test_X)
    
    all_predictions_full.append(predictions.numpy())
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((preds_full - actuals_full) ** 2)
print(f'Full Data MSE: {mse_full:.6f} (n={len(preds_full)})')


=== Full Data CV (Importance-Weighted) ===


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [01:40<20:03, 100.31s/it]

 15%|█▌        | 2/13 [03:15<17:53, 97.57s/it] 

 23%|██▎       | 3/13 [04:48<15:52, 95.27s/it]

...

 85%|████████▍ | 11/13 [16:41<03:07, 93.93s/it]

 92%|█████████▏| 12/13 [18:24<01:36, 96.87s/it]

100%|██████████| 13/13 [19:57<00:00, 95.54s/it]

100%|██████████| 13/13 [19:57<00:00, 92.11s/it]

Full Data MSE: 0.010802 (n=1227)


In [10]:
# Calculate overall MSE
n_single = len(preds_single)
n_full = len(preds_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE SUMMARY (Importance-Weighted) ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_030): 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')

# Predict LB using CV-LB relationship
predicted_lb = 4.22 * overall_mse + 0.0534
print(f'\nPredicted LB (using LB = 4.22*CV + 0.0534): {predicted_lb:.4f}')
print(f'Best LB achieved: 0.0877')
print(f'Target: 0.0347')


=== CV SCORE SUMMARY (Importance-Weighted) ===
Single Solvent MSE: 0.009723 (n=656)
Full Data MSE: 0.010802 (n=1227)
Overall MSE: 0.010426

Best CV (exp_030): 0.008194

✗ WORSE: 27.24% worse than best CV

Predicted LB (using LB = 4.22*CV + 0.0534): 0.0974
Best LB achieved: 0.0877
Target: 0.0347


In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ImportanceWeightedEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y, X_test=test_X)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:21, 21.08s/it]

2it [00:41, 20.68s/it]

3it [00:59, 19.55s/it]

4it [01:18, 19.33s/it]

...

21it [07:05, 20.10s/it]

22it [07:24, 19.94s/it]

23it [07:47, 20.74s/it]

24it [08:06, 20.33s/it]

24it [08:06, 20.28s/it]

In [12]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ImportanceWeightedEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y, X_test=test_X)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [01:40, 100.22s/it]

2it [03:15, 97.56s/it] 

3it [04:48, 95.46s/it]

4it [06:10, 89.84s/it]

...

10it [15:02, 91.85s/it]

11it [16:41, 94.13s/it]

12it [18:25, 97.03s/it]

13it [19:57, 95.66s/it]

13it [19:57, 92.14s/it]

In [13]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [14]:
# Final verification
print(f'\n=== FINAL CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_030): 0.008194')
print(f'Predicted LB: {predicted_lb:.4f}')
print(f'Target: 0.0347')


=== FINAL CV SCORE ===
Single Solvent MSE: 0.009723 (n=656)
Full Data MSE: 0.010802 (n=1227)
Overall MSE: 0.010426

Best CV (exp_030): 0.008194
Predicted LB: 0.0974
Target: 0.0347
